In [ ]:
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from transformers import BertTokenizer,BertForSequenceClassification,Trainer,TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive
!pip install datasets
!pip install evaluate
from datasets import Dataset
import evaluate
from scipy.special import expit
#
drive.mount('/content/drive')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#
windows = pd.read_csv('/content/drive/MyDrive/Saves/windows.csv',index_col=0)#index_col=0 agnoei thn unamed sthlh

print(windows.columns)
print('Class distribution: ',windows['labels'].value_counts())
#
print('Null text: ',windows['text'].isnull().sum())
print('Null labels: ',windows['labels'].isnull().sum())
#
windows['text'] = windows['text'].fillna('')
print('Null text: ',windows['text'].isnull().sum())

#
#input('WAIT')
##
windows_train,windows_test = train_test_split(windows,test_size=0.1,random_state=42,shuffle=True)
##
windows_train,windows_validation = train_test_split(windows_train,test_size=0.1111,random_state=42,shuffle=True)
######Dataframe to HuggingFace Dataset#####

windows_train = Dataset.from_pandas(windows_train)
windows_test = Dataset.from_pandas(windows_test)
windows_validation = Dataset.from_pandas(windows_validation)
###########
###TOKENIZER && TOKENIZING####
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") # tokenizer

def tokenize_logs(entry):
  tokens = tokenizer(entry['text'],padding='max_length',truncation=True)
  #tokens['labels'] = entry['Label']
  return tokens
###TOKENIZING SUB DATASETS######
windows_train = windows_train.map(tokenize_logs,batched=True)
windows_test = windows_test.map(tokenize_logs,batched=True)
windows_validation = windows_validation.map(tokenize_logs,batched=True)

##METRICS####
accuracy = evaluate.load("accuracy")
other_metrics = evaluate.combine(["precision","recall","f1"])
roc_auc = evaluate.load("roc_auc")
#combine all metrics to a dictionary
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    prediction_scores = expit(predictions)#expit(logits)
    roc_auc_score = roc_auc.compute(references=labels,prediction_scores=prediction_scores[:,1])#y_true,y_score
    predictions = np.argmax(predictions, axis=-1)
    other_metrics_scores = other_metrics.compute(predictions=predictions,references=labels,average=None)#kathe klash
    accuracy_score = accuracy.compute(predictions=predictions,references=labels)["accuracy"]#epistrefei dict kai perno thn timh tou accuracy:
    return {"accuracy":accuracy_score,"precision_class_0":other_metrics_scores["precision"][0],"precision_class_1":other_metrics_scores["precision"][1],"recall_class_0":other_metrics_scores["recall"][0],"recall_class_1":other_metrics_scores["recall"][1],"f1_class_0":other_metrics_scores["f1"][0],"f1_class_1":other_metrics_scores["f1"][1],"roc_auc":roc_auc_score["roc_auc"]}

###METRICS#####

###MODEL###
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2).to(device)#modelo me ena extra layer gia to classification
###TRAINING ARGS#####
training_arguments = TrainingArguments(
    output_dir = "/content/drive/MyDrive/Run",#pou tha apothikeytei to modelo kai ta checkpoints. gia persistency to vazo sto /content/MyDrive/...
    overwrite_output_dir=True,  # MONO GIA THN PROTH FORA META PREPEI FALSE
    eval_strategy = "steps",#to evaluation ginetai sto telos kathe synolou steps
    learning_rate=2e-5,#rythmos mathisis genika na einai mikros xoris akrotites. mikrh timh-> kalyterh genikeysh
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,#genika de thelei oute poly liga alla oute polla.poses fores to modelo tha iterarei to training dataset-koinos se posa epochs tha ginei to training
    weight_decay=0.01,#meiosh tou overfitting
    resume_from_checkpoint=True,
    save_steps=500,  # Save model every 500 steps
    save_total_limit=2  # Keep only the last 2 checkpoints to save disk space
    #kapoia akoma orismata + fakelos
)
####TRAINER###
trainer = Trainer(
    model=model,#modelo pou tha kanei to classification,
    args=training_arguments,#ta orismata tis ekpaideyshs apo pano
    #tokenizer=tokenizer#o tokenizer, an exo kanei hdh preprocessing-tokenizing LOGIKA den to xreiazomai
    train_dataset=windows_train,#to synolo ekpaideyshs gia to fine-tuning
    eval_dataset=windows_validation,#to synolo elegxou-aksiologisis gia to fine tuning-xoris ayto de tha aksiologithei h apodosi tou modelou mono tha ekpaideytei
    #tokenizer=tokenizer
    compute_metrics=compute_metrics#exei os apotelesma sto training tou modelou ektos apo to training/validation loss na emfanizontai kai oi times ton metrikon pou orisa sth synartisi kai epistrefontai apo ayth
    #
    )

####TRAIN && EVALUATE
trainer.train()# to orisma RESUME benei AFOU ginei interrupt!-to resume-from-checkpoint=directory
results = trainer.evaluate(eval_dataset=windows_test) #ti apodosi fernei to modelo sto test dataset-orisma to test dataset
print(results)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
Mounted at /content/drive
cuda
Index(['Node', 'Window Start', 'text', 'labels'], d

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy,Precision Class 0,Precision Class 1,Recall Class 0,Recall Class 1,F1 Class 0,F1 Class 1,Roc Auc
500,0.043100,0.010783,0.998219,0.998132,0.999369,0.999952,0.975956,0.999041,0.987523,0.999473
1000,0.006200,0.010978,0.998219,0.998084,1.000000,1.000000,0.975339,0.999041,0.987516,0.999608
1500,0.007300,0.002748,0.999555,0.999712,0.997531,0.999808,0.996301,0.999760,0.996915,0.999849
2000,0.005200,0.002256,0.999555,0.999808,0.996305,0.999712,0.997534,0.999760,0.996919,0.999991
2500,0.003700,0.001178,0.999777,0.999808,0.999382,0.999952,0.997534,0.999880,0.998457,0.999997
3000,0.002200,0.001248,0.999822,0.999808,1.000000,1.000000,0.997534,0.999904,0.998765,0.999999
3500,0.001400,0.001265,0.999599,0.999856,0.996308,0.999712,0.998150,0.999784,0.997228,0.999999
4000,0.001200,0.002217,0.999733,0.999760,0.999382,0.999952,0.996917,0.999856,0.998148,0.999996
4500,0.002300,0.001272,0.999688,0.999808,0.998149,0.999856,0.997534,0.999832,0.997842,0.999996
5000,0.004500,0.000411,0.999777,0.999856,0.998766,0.999904,0.998150,0.999880,0.998458,1.000000
